In [1]:
import numpy as np
import random
import math
import sys

import NBS

np.set_printoptions(precision=2)

In [2]:
# U[0 1] Distribution
# Takes num items
# Returns np.ndarray of shape (num_items,) of values drawn from U[0 1]
def uni_0_1(num_items):
    bids = np.zeros(num_items)
    for b in range(num_items):
        bids[b] = np.random.uniform(0,1)
    return bids

In [3]:
# Square Root Distribution
# Takes num items
# Returns np.ndarray of shape (num_items,) of values drawn from Square Root Distribution
def sqrt(num_items):
    bids = np.zeros(num_items)
    for b in range(num_items):
        bids[b] = (np.random.uniform(0,1))**2
    return bids

In [4]:
# Takes np.ndarray of values for n agents
# Returns 2 np.ndarrays and list: values for selected agents, values for non-selected agents
# ciel(n/2) selected agents, floor(n/2) non-selected agents, indices of selected agents
def random_subset(vals, selected_indices):
    if type(vals) != np.ndarray:
        raise Exception("vals is not type np.ndarray, but type: ", type(vals))
    if type(selected_indices) != list:
        raise Exception("selected_indices is not type list, but type: ", type(selected_indices))
    
    num_selected = int(math.ceil(float((vals.shape[0] - len(selected_indices)) / 2)))
    new_indices = []
    while len(new_indices) != num_selected:
        r = random.randint(0, vals.shape[0]-1)
        if r not in selected_indices:
            selected_indices.append(r)
            new_indices.append(r)
    
    selected_agents = np.zeros(shape=(num_selected, vals.shape[1]))
#     nonselected_agents = np.zeros(shape=(vals.shape[0]-num_selected, vals.shape[1]))
    
    selected_index = 0
#     nonselected_index = 0
    
    for i in range(vals.shape[0]):
        if (i in new_indices):
            selected_agents[selected_index] = vals[i]
            selected_index += 1
#         else:
#             nonselected_agents[nonselected_index] = vals[i]
#             nonselected_index += 1
#     return selected_agents, nonselected_agents, selected_indices
    return selected_agents, new_indices

In [1]:
def PA(selected, cap):
    deleted = []
    # Checking for indifferent agents and removing them from selected
    for i in range(selected.shape[0]):
        avg = np.average(selected[i])
        avg_array = np.zeros(selected.shape[0].size)
        for j in range(selected.shape[0]):
            avg_array[j] = avg
        if np.allclose(selected[i], avg_array):
            selected = np.delete(selected, i, axis=0)
            deleted.append(i)
    
    # Calculating outside option
    O = np.zeros(selected.shape[0])
    for i in range(O.size):
        O[i] = np.sum(np.multiply(selected[i], cap)) / selected.shape[1]

    # Nash Social Welfare Optimal Probability Matrix
    nsw = NBS.NBS(selected, O, cap)
    if nsw is None:
        raise Exception("NBS returned None.")

    # NSW Optimal Utility
    util = np.sum(np.multiply(nsw, selected), axis=1)
    
    # Calculating f for each agent
    f = np.zeros(selected.shape[0])
    for i in range(f.size):
        new_sel = np.delete(selected, i, axis=0)
        new_O = np.delete(O, i)
        
        i_exclusive = NBS.NBS(new_sel, new_O, cap)
        if i_exclusive is None:
            raise Exception("NBS returned None.")

        new_util = np.sum(np.multiply(new_sel, i_exclusive), axis=1)


        num = 1
        denom = 1
        for j in range(i):
            num *= util[j]
            denom *= new_util[j]
        if i < new_util.shape[0]:
            num *= util[i]
            for j in range(i+1, util.shape[0]):
                    num *= util[j]
                    denom *= new_util[j-1]
    
        f[i] = num/denom
    
    # Applying f to each agent
    probs = nsw
    for i in range(probs.shape[0]):
        probs[i] *= f[i]
    # FIGURE OUT HOW BEST TO NOT CONSIDER INDIFFERENT AGENTS WHEN DOING NBS BUT TO ADD THEM BACK AT END
    return probs

In [35]:
def pref_att(num_agents, num_items, p):
    array = np.zeros(shape=(num_agents, num_items))
    array[0] = np.random.rand(num_items)
    for i in range(1, num_agents):
        rint = np.random.randint(0, i)
        array[i] = array[rint]
        for j in range(1, num_items):
            r = np.random.rand()
            if r < p:
                array[i][j] = np.random.rand()
    return array

In [69]:
anon = pref_att(4,8,0.5)
pa = PA(anon, np.ones(8))
print("PA: \n", pa)
print("Agent Sums: \n", np.sum(pa, axis=1))
print("Item Sums: \n", np.sum(pa, axis=0))
print("Total Sums: ", np.sum(pa))

PA: 
 [[0.00e+00 0.00e+00 3.57e-10 0.00e+00 7.70e-01 1.96e-09 0.00e+00 5.86e-09]
 [0.00e+00 3.35e-09 5.00e-10 0.00e+00 1.07e-09 3.87e-09 0.00e+00 7.70e-01]
 [1.98e-11 0.00e+00 7.34e-10 9.18e-01 1.41e-10 2.39e-09 1.30e-10 3.49e-10]
 [0.00e+00 1.00e+00 0.00e+00 1.11e-10 1.11e-09 6.26e-09 7.01e-11 3.87e-10]]
Agent Sums: 
 [0.77 0.77 0.92 1.  ]
Item Sums: 
 [1.98e-11 1.00e+00 1.59e-09 9.18e-01 7.70e-01 1.45e-08 2.00e-10 7.70e-01]
Total Sums:  3.4573526137108512


In [86]:
# Takes 2-D np.ndarray vals matrix, 1-D np.ndarray course caps matrix, and int of agents left to stop recursing
# Returns 2-D np.ndarray Probability Distribution
def RPI_recurse(vals, selected_indices, cap, n_knot=4):
    #---------------------------------------------INVARIANT TESTS------------------------------------------
    if type(vals) != np.ndarray:
        raise Exception("vals must be type np.ndarray. Current type: ", type(vals))
    if type(selected_indices) != list:
        raise Exception("selected_indices must be type list. Current type: ", type(selected_indices))
    if type(cap) != np.ndarray:
        raise Exception("cap must be type np.ndarray. Current type: ", type(cap))
    if type(n_knot) != int:
        raise Exception("n_knot must be type int. Current type: ", type(n_knot))
    
    if np.ndim(vals) != 2:
        raise Exception("vals must be a 2-D np.ndarray. Current shape: ", vals.shape)
    if np.ndim(cap) != 1:
        raise Exception("cap must be a 1-D np.ndarray. Current shape: ", cap.shape)
        
    if n_knot < 4:
        raise Exception("n_knot must be >= 4. Current n_knot: ", n_knot)
        
    #-----------------------------------------------TESTS END------------------------------------------------
        
    # Algorithm Start:
    n_hat = vals.shape[0]-len(selected_indices)
    if n_hat < n_knot:
        P = np.zeros(shape=vals.shape)
        uni_probs = cap / n_hat
        for i in range(P.shape[0]):
            if i not in selected_indices:
                P[i] = uni_probs
        return P
    else:
        # Seperate half of agents randomly
        selected, new_indices = random_subset(vals, selected_indices)
        
        # Partial Allocation Mechanism
        P_selected = PA(selected, cap)

        
        # Tweaking PA Mechanism Output
        total_alloc = np.sum(P_selected, axis=1)

        for i in range(P_selected.shape[0]):
            first_part = (1.0-float(total_alloc[i]/2))
            for j in range(P_selected.shape[1]):
                second_part = cap[j]/(n_hat)
                P_selected[i][j] = float(P_selected[i][j] / 2) + first_part*second_part

        # Recursively calling RPI_recurse
        cap = cap - np.sum(P_selected, axis=0)
        
        P = np.zeros(shape=vals.shape)
        for i in range(len(new_indices)):
            P[new_indices[i]] = P_selected[i]
        return np.add(RPI_recurse(vals, selected_indices, cap, n_knot),  P)

In [84]:
for i in range(4,33):   
    array = pref_att(i,i,0.5)
    nb = RPI_recurse(array, [], np.ones(i))
    print("Size: ", i)
    print("P: \n", nb)
    if nb is not None:
                if not np.allclose(np.sum(nb, axis=0), np.ones(i)):
                    print("Not Doubly Stochastic along axis 0. Item Sums: \n", np.sum(nb, axis=0))
                if not np.allclose(np.sum(nb, axis=1), np.ones(i)):
                    print("Not Doubly Stochastic along axis 1. Agent Sums: \n", np.sum(nb, axis=1))
                if np.allclose(np.sum(nb, axis=0), np.ones(i)):
                    if np.allclose(np.sum(nb, axis=1), np.ones(i)):
                        print("Doubly Stochastic")
                print("Sum: ", np.sum(nb))
                print("___________________________________\n")

Size:  4
P: 
 [[0.37 0.12 0.13 0.37]
 [0.13 0.62 0.13 0.13]
 [0.13 0.13 0.61 0.13]
 [0.37 0.12 0.13 0.37]]
Doubly Stochastic
Sum:  4.0
___________________________________

Size:  5
P: 
 [[0.11 0.55 0.11 0.11 0.11]
 [0.12 0.12 0.54 0.12 0.12]
 [0.1  0.1  0.1  0.1  0.6 ]
 [0.34 0.12 0.12 0.34 0.09]
 [0.34 0.12 0.12 0.34 0.09]]
Doubly Stochastic
Sum:  5.0
___________________________________

Size:  6
P: 
 [[0.12 0.4  0.12 0.12 0.12 0.12]
 [0.1  0.1  0.1  0.1  0.1  0.52]
 [0.23 0.14 0.23 0.23 0.08 0.09]
 [0.23 0.14 0.23 0.23 0.08 0.09]
 [0.23 0.14 0.23 0.23 0.08 0.09]
 [0.09 0.09 0.09 0.09 0.54 0.09]]
Doubly Stochastic
Sum:  6.0
___________________________________

Size:  7
P: 
 [[0.09 0.17 0.09 0.09 0.09 0.38 0.09]
 [0.21 0.19 0.21 0.09 0.11 0.1  0.09]
 [0.21 0.19 0.21 0.09 0.11 0.1  0.09]
 [0.21 0.19 0.21 0.09 0.11 0.1  0.09]
 [0.07 0.07 0.07 0.07 0.18 0.07 0.46]
 [0.09 0.09 0.09 0.46 0.09 0.09 0.09]
 [0.11 0.11 0.11 0.11 0.3  0.16 0.11]]
Doubly Stochastic
Sum:  7.0
_____________________

Size:  17
P: 
 [[0.23 0.04 0.1  0.04 0.05 0.02 0.06 0.02 0.01 0.04 0.05 0.06 0.08 0.04
  0.03 0.04 0.08]
 [0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
  0.04 0.4  0.04]
 [0.04 0.36 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
  0.04 0.04 0.04]
 [0.09 0.09 0.04 0.07 0.02 0.05 0.13 0.05 0.01 0.02 0.02 0.09 0.05 0.07
  0.01 0.02 0.17]
 [0.04 0.04 0.04 0.42 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
  0.04 0.04 0.04]
 [0.04 0.02 0.02 0.02 0.02 0.01 0.04 0.38 0.01 0.04 0.08 0.04 0.04 0.04
  0.1  0.02 0.04]
 [0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
  0.45 0.03 0.03]
 [0.05 0.03 0.03 0.02 0.02 0.01 0.05 0.05 0.1  0.31 0.02 0.05 0.05 0.05
  0.06 0.03 0.05]
 [0.06 0.03 0.03 0.03 0.01 0.02 0.05 0.02 0.04 0.14 0.01 0.03 0.29 0.03
  0.01 0.14 0.06]
 [0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.47 0.03 0.03 0.03
  0.03 0.03 0.03]
 [0.03 0.03 0.03 0.03 0.03 0.53 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
  0.03 0.03 0

Size:  22
P: 
 [[0.04 0.01 0.04 0.01 0.01 0.26 0.04 0.01 0.24 0.01 0.02 0.02 0.04 0.01
  0.01 0.04 0.02 0.04 0.04 0.02 0.04 0.04]
 [0.12 0.01 0.11 0.02 0.01 0.02 0.05 0.01 0.02 0.04 0.02 0.05 0.05 0.01
  0.01 0.31 0.02 0.02 0.02 0.02 0.02 0.05]
 [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.52 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.02 0.02 0.02 0.02 0.51 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.13 0.03 0.02 0.09 0.01 0.11 0.06 0.01 0.02 0.01 0.02 0.01 0.06 0.01
  0.01 0.06 0.02 0.12 0.02 0.11 0.02 0.06]
 [0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03 0.03
  0.03 0.03 0.47 0.03 0.03 0.03 0.03 0.03]
 [0.02 0.51 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.04 0.01 0.04 0.01 0.01 0.13 0.04 0.03 0.04 0.01 0.01 0.02 0.04 0.12
  0.01 0.04 0.01 0.08 0.04 0.01 0.22 0.04]
 [0.06 0.01 0.02 0.02 0.02 0.02 0.17 0.02 0.07 0.01 0.07 0.01 0.1

Size:  25
P: 
 [[0.19 0.08 0.06 0.02 0.01 0.06 0.02 0.01 0.02 0.04 0.02 0.01 0.04 0.09
  0.02 0.02 0.06 0.02 0.06 0.01 0.02 0.03 0.04 0.01 0.06]
 [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.49 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.04 0.02 0.06 0.02 0.01 0.06 0.02 0.01 0.02 0.01 0.02 0.03 0.02 0.02
  0.02 0.08 0.06 0.02 0.16 0.04 0.02 0.01 0.19 0.01 0.06]
 [0.02 0.02 0.02 0.02 0.51 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.16 0.01 0.03 0.01 0.01 0.03 0.01 0.01 0.03 0.01 0.01 0.01 0.01 0.01
  0.03 0.03 0.03 0.03 0.03 0.01 0.37 0.01 0.03 0.01 0.03]
 [0.02 0.5  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.51 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.51 0.02
  0

Size:  28
P: 
 [[0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.43 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.04 0.   0.04 0.01 0.02 0.29 0.02 0.01 0.04 0.04 0.06 0.01 0.04 0.01
  0.05 0.02 0.03 0.01 0.   0.03 0.04 0.01 0.01 0.11 0.04 0.01 0.01 0.01]
 [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.49 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.03 0.01 0.01 0.01 0.01 0.03 0.01 0.37 0.03 0.03 0.02 0.01 0.03 0.01
  0.01 0.03 0.16 0.01 0.01 0.02 0.01 0.01 0.03 0.03 0.03 0.03 0.01 0.03]
 [0.03 0.01 0.01 0.01 0.01 0.03 0.01 0.03 0.03 0.03 0.02 0.01 0.03 0.04
  0.01 0.32 0.03 0.11 0.01 0.03 0.01 0.01 0.03 0.03 0.03 0.03 0.01 0.03]
 [0.02 0.02 0.02 0.02 0.32 0.02 0.02 0.02 0.02 0.02 0.03 0.02 0.02 0.15
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02]
 [0.05 0.01 0.01 0.03 0.02 0.05 0.02 0.05 0.05 0.05 0.02 0.01 0.25 0.01
  0.01 0.12 0.04 0.01 0.03 0.04 0.01 0.01 0

Size:  30
P: 
 [[0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.52]
 [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.5  0.02 0.02 0.02 0.02
  0.02 0.02]
 [0.03 0.01 0.01 0.37 0.01 0.01 0.03 0.03 0.03 0.01 0.03 0.03 0.03 0.03
  0.03 0.03 0.03 0.01 0.01 0.15 0.03 0.03 0.01 0.01 0.01 0.01 0.01 0.01
  0.01 0.01]
 [0.04 0.01 0.01 0.09 0.01 0.04 0.04 0.02 0.04 0.01 0.02 0.04 0.24 0.01
  0.04 0.04 0.02 0.01 0.03 0.03 0.13 0.01 0.01 0.01 0.01 0.01 0.01 0.01
  0.02 0.01]
 [0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02
  0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.02 0.51 0.02
  0.02 0.02]
 [0.03 0.01 0.01 0.03 0.01 0.01 0.03 0.03 0.03 0.11 0.03 0.03 0.03 0.03
  0.03 0.03 0.03 0.01 0.01 0.03 0.03 0.03 0.01 0.11 0.01 0.01 0.11 0.01
  0.01 0.11]
 [0.11 0.01 0.01 0.01 0.01 0.01 0.03 0.05 0

SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [10]:
def random_generator(num_agents, num_items):
    array = np.random.rand(num_agents * num_items)
    array.shape = (num_agents, num_items)
    return array

In [81]:
for i in range(4, 10):
    for j in range(4,10):
#         print("Matrix Size: (",i, ",", j,")")
        a = pref_att(i,j, 0.5)
#         print("array: \n", a)
        b = RPI_recurse(a, [], np.ones(j))
#         print("Probability Distribution: \n", b)
        if b is not None:
            if not (np.allclose(np.sum(b, axis=0), np.ones(j))) and (np.allclose(np.sum(b, axis=1), np.ones(i))):
                raise Exception("Not Doubly Stochastic. Size: ",i," ,",j)
#         print("Prob Dist Agent Sums: ", np.sum(b, axis=1))
#         print("Prob Dist Item Sums: ", np.sum(b, axis=0))
#         print("\n _____________________________")

Exception: NBS returned None.

In [ ]:
# sys.eps instead of np.zeros